In [4]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# Import opendatasets library to download dataset directly from kaggle
import opendatasets as od
import pandas as pd

od.download("https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: rameshdkoppad
Your Kaggle Key: ··········


100%|██████████| 7.86M/7.86M [00:00<00:00, 23.4MB/s]


For reference:

https://www.geeksforgeeks.org/how-to-download-kaggle-datasets-into-jupyter-notebook/

In [6]:
# Import dependencies/libraries required
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
import string
import re

In [28]:
from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer, PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [8]:
# reading the CSV file
file =('ecommerce-text-classification/\
ecommerceDataset.csv')
dataset = pd.read_csv(file)
dataset

,Household,"Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and some for eternal bliss.so bring home this elegant print that is lushed with rich colors that makes it nothing but sheer elegance to be to your friends and family.it would be treasured forever by whoever your lucky recipient is. Liven up your place with these intriguing paintings that are high definition hd graphic digital prints for home, office or any room."
0,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
1,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
2,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
3,Household,Incredible Gifts India Wooden Happy Birthday U...
4,Household,Pitaara Box Romantic Venice Canvas Painting 6m...
...,...,...
50419,Electronics,Strontium MicroSD Class 10 8GB Memory Card (Bl...
50420,Electronics,CrossBeats Wave Waterproof Bluetooth Wireless ...
50421,Electronics,Karbonn Titanium Wind W4 (White) Karbonn Titan...
50422,Electronics,"Samsung Guru FM Plus (SM-B110E/D, Black) Colou..."


In [9]:
# Set the column hedings
dataset.columns = ['label', 'text']

In [10]:
# Top rows of dataset
dataset.head()

,label,text
0,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
1,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
2,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
3,Household,Incredible Gifts India Wooden Happy Birthday U...
4,Household,Pitaara Box Romantic Venice Canvas Painting 6m...


In [11]:
# Rows and Columns of dataset
dataset.shape

(50424, 2)

In [12]:
# Shows rangeindex, columns along with count of Non-null values, datatype
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50424 entries, 0 to 50423
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   50424 non-null  object
 1   text    50423 non-null  object
dtypes: object(2)
memory usage: 788.0+ KB


In [13]:
# 'value_counts' returns the count of labels column 
dataset['label'].value_counts()

Household                 19312
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: label, dtype: int64

In [14]:
# Here the 'Household' products are more compared to others. so, there might be a problem of class imbalance in dataset

# Now let's encode the labels
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
dataset['label_encoded'] = labelencoder.fit_transform(dataset['label'])

In [15]:
dataset[['label', 'label_encoded']].value_counts()

label                   label_encoded
Household               3                19312
Books                   0                11820
Electronics             2                10621
Clothing & Accessories  1                 8671
dtype: int64

In [16]:
# Storing the label and text in y and x
X = dataset['text']
y = dataset['label_encoded']

In [17]:
# Splitting the dataset using stratified shuffle splitting
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.4, random_state = 42)
sss.get_n_splits(X, y)

1

In [18]:
for train_index, test_index in sss.split(X, y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

Preprocessing functions

In [19]:
# Some preprocessing functions for 'text' column
# function to convert the text to lower case

def convert_to_lowercase(text):
  return text.str.lower()

# function to remove punctuations from the text
def remove_punctuations(text):
  english_punctuation = string.punctuation
  translator = str.maketrans('', '', english_punctuation)
  return str(text).translate(translator)

# function to remove stopwords from the text
def remove_stopwords(text):
  from nltk.corpus import stopwords
  stopwords = set(stopwords.words('english'))
  return " ".join([word for word in str(text).split() if word not in stopwords])

# function to remove repeating characters
def remove_repeating_characters(text):
  return re.sub(r'(.)1+', r'1', text)

# function to remove numeric text
def remove_numeric(text):
  return re.sub('[0-9]+', '', text)

# Tokenizing the text
def tokenize_text(text):
  tokenizer = RegexpTokenizer('\w+')
  text = text.apply(tokenizer.tokenize)
  return text

# Lemmatizing the text. i.e, Converting some of the words to their root form
def text_lemmatization(text):
  lem = WordNetLemmatizer()
  text = [lem.lemmatize(word) for word in text]
  return text

In [24]:
# Creating a preprocess function. this function will help in both training the data and also when we will predict a
# single example from the trained model. i.e, inferencing.

def preprocess(text):
  text = convert_to_lowercase(text)
  text = text.apply(lambda x : remove_punctuations(x))
  text = text.apply(lambda x : remove_stopwords(x))
  text = text.apply(lambda x : remove_repeating_characters(x))
  text = text.apply(lambda x : remove_numeric(x))
  text = tokenize_text(text)
  text = text.apply(lambda x : text_lemmatization(x))
  text = text.apply(lambda x: " ".join(x))
  
  return text

In [29]:
# Let's preprocess the training data
X_train = preprocess(X_train)

In [30]:
X_train

26636                         trump think like billionaire
37698    newmom maternity legging seamless tummy suppor...
22143         ugc netjrf exam solved paper psychology book
12302    masoom nx heavy base clear shot glass vodka te...
14098    nextgeek ac steam generator iron power cable p...
                               ...                        
4694     polyworm bernini wall oval mirror pwrmlargelav...
30728    feeling good new mood therapy review book read...
17953    blue heaven studio primer g makeup fixer mist ...
9608     premium quality heavy tadka panfrying panwagha...
25043    everything kid learning french book fun exerci...
Name: text, Length: 30254, dtype: object

In [35]:
# Transforming the training data using the TF-IDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range= (1,2), max_features= 300000)
vectorizer.fit(X_train)

TfidfVectorizer(max_features=300000, ngram_range=(1, 2))

In [36]:
# No of feature words
print("No. of Feature words: ", len(vectorizer.get_feature_names()))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


No. of Feature words:  300000


In [37]:
# Transforming the training data using the vectorizer
X_train = vectorizer.transform(X_train)

In [38]:
# Model
# Here i'm trying two different models one Random forest classifier and another XGboost classifier

from random import Random
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

random_forest_classifier = RandomForestClassifier(n_estimators= 10, criterion= 'entropy', random_state= 42)
random_forest_classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [39]:
# XGBoost model
xgb_boost_classifier = XGBClassifier()
xgb_boost_classifier.fit(X_train, y_train)

XGBClassifier(objective='multi:softprob')

In [40]:
# Now we will tes the two models on the test set with accuracy_score metric.
from sklearn.metrics import accuracy_score

X_test = preprocess(X_test) # we have preprocess the test data too
X_test = vectorizer.transform(X_test) #transforming the testing data
random_forest_predictions = random_forest_classifier.predict(X_test)
xgb_predictions = xgb_boost_classifier.predict(X_test)
random_forest_score = accuracy_score(y_test, random_forest_predictions)
xgb_score = accuracy_score(y_test, xgb_predictions)

print("Random Forest Classifier Accuracy: ", random_forest_score * 100 , "%")
print("XGBoost Classifier Accuracy: ", xgb_score * 100 , "%")

Random Forest Classifier Accuracy:  95.25532969757064 %
XGBoost Classifier Accuracy:  87.96232027764006 %


In [44]:
# Both models are giving pretty good accuracies but we will choose xgboost as the final model.
# Now to make prediction of a single example we will write a function.

def predict_custom(inp):
    inp = pd.Series(inp)
    inp = preprocess(inp)
    inp = [inp[0],]
    inp = vectorizer.transform(inp)
    prediction = xgb_boost_classifier.predict(inp)
    prediction = prediction[0]

    interpretations = {
        0 : "Electronics",
        1 : "Households", 
        2 : "Books",
        3 : "Clothing and Accessories"
    }

    for i in interpretations.keys():
        if i == prediction:
            return interpretations[i]

In [47]:
# lets try some random examples from the data

# Electronics
inp1 = "A Dance with Dragons: A Song of Ice and Fire: Book Five Review “Filled with vividly rendered set pieces, unexpected turnings, assorted cliffhangers and moments of appalling cruelty, A Dance with Dragons is epic fantasy as it should be written: passionate, compelling, convincingly detailed and thoroughly imagined.”—The Washington Post   “Long live George Martin . . . a literary dervish, enthralled by complicated characters and vivid language, and bursting with the wild vision of the very best tale tellers.”—The New York Times   “One of the best series in the history of fantasy.”—Los Angeles Times  About the Author George R.R. Martin sold his first story in 1971 and has been writing professionally since then. He spent ten years in Hollywood as a writer-producer, working on The Twilight Zone, Beauty and the Beast, and various feature films and television pilots that were never made. In the mid '90s he returned to prose, his first love, and began work on his epic fantasy series, A Song of Ice and Fire. He has been in the Seven Kingdoms ever since. Whenever he's allowed to leave, he returns to Santa Fe, New Mexico, where he lives with the lovely Parris, a big white dog called Mischa, and two cats named Augustus and Caligula, who think they run the place. 				    	 					              See all Product description"
print("prediction inp1: ", predict_custom(inp1))

# Households
inp2 = "KEZRO Men's Wallet (Brown)"
print("prediction inp2: ", predict_custom(inp2))

prediction inp1:  Electronics
prediction inp2:  Households


In [49]:
# books
inp3= "Mushkiya ABD-026 C-Cut Double Layer Abaya burkha for women girl About Mushkiya Mushkiya is an international islamic clothing brand that represents style and modest clothing since 2006. Founded in Dubai , Mushkiya works on a basis of creativity and responsibility, fused with a modesty of islamic clothing.Fashion with modesty are at the heart of the brand’s philosophy. Product Description Simple yet trendy Abaya Dress in double layer and frills on shoulders would become your favourite in no time. This whimsical 'C-Cut' Abaya Dress Is made of Nida-Matt fabric which is soft, breathable and has a great fall. Pinping on the front layer, enhances the look of the Abaya. This Abaya can also be used as a Dress or a Burqa if you prefer colorful ones."
print("prediction inp3: ", predict_custom(inp3))

# Clothing and Accessories
inp4 = "Sony DVD Printable Blanks with White Surface -Jar Pack of 100 This product is a must have for many around the world. As it gives it's users the experience they truly desire for."
print("prediction inp4: ", predict_custom(inp4))

prediction inp3:  Households
prediction inp4:  Clothing and Accessories
